In [1]:
import tensorflow as tf
import numpy as np

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data(path='mnist.npz')


In [3]:
# 55000 data samples
print("Train Samples: {}".format(x_train.shape))
print("Train Outputs: {}".format(y_train.shape))

# Sample size 28x28
# 10000 test samples
print("Eval Samples: {}".format(x_test.shape))
print("Eval Outputs: {}".format(y_test.shape))

Train Samples: (60000, 28, 28)
Train Outputs: (60000,)
Eval Samples: (10000, 28, 28)
Eval Outputs: (10000,)


In [18]:
x_train = x_train.astype(float)
y_train = y_train.astype(np.int32)

In [19]:
type(x_train)

numpy.ndarray

In [20]:
type(x_train[0][0][0])

numpy.float64

In [21]:
type(y_train)

numpy.ndarray

In [22]:
type(y_train[0])

numpy.int32

In [23]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
            inputs=input_layer,
            filters=32,
            kernel_size=[5, 5],
            padding="same",
            activation=tf.nn.relu)

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2
    conv2 = tf.layers.conv2d(
            inputs=pool1,
            filters=64,
            kernel_size=[5, 5],
            padding="same",
            activation=tf.nn.relu)

    # Pooling Layer #2
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Dense Layer with Dropout
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=10)

    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    # Compute evaluation metrics.
    accuracy = tf.metrics.accuracy(labels=labels, predictions= predictions["classes"], name='acc_op')
    metrics = {'accuracy': accuracy}
    # merging and saving them every 100 steps by default for Tensorboard
    tf.summary.scalar('accuracy', accuracy[1])

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    tf.summary.scalar('loss', loss)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {"accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)



In [24]:
# Create the Estimator
mnist_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn,
                                          model_dir="./checkpoints/log1",
                                          config=tf.estimator.RunConfig(log_step_count_steps=50,
                                                                        save_summary_steps=50),
                                         )

INFO:tensorflow:Using config: {'_model_dir': './checkpoints/log1', '_tf_random_seed': None, '_save_summary_steps': 50, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 50, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000200A8490860>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [25]:
# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=10)

# Prepare input function
train_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x": x_train},
                                                    y= y_train ,
                                                    batch_size=100,
                                                    num_epochs=None,
                                                    shuffle=True)

In [26]:
# Train the model
mnist_classifier.train(input_fn=train_input_fn,
                       steps=10,
                       hooks=[logging_hook])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./checkpoints/log1\model.ckpt.


NotFoundError: No registered '_MklConv2DWithBias' OpKernel for CPU devices compatible with node conv2d/BiasAdd = _MklConv2DWithBias[T=DT_DOUBLE, _kernel="MklOp", data_format="NHWC", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:CPU:0"](Reshape, conv2d/kernel/read, conv2d/bias/read, DMT/_0, DMT/_1, DMT/_2)
	 (OpKernel was found, but attributes didn't match)
	.  Registered:  device='CPU'; label='MklOp'; T in [DT_FLOAT]

	 [[Node: conv2d/BiasAdd = _MklConv2DWithBias[T=DT_DOUBLE, _kernel="MklOp", data_format="NHWC", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:CPU:0"](Reshape, conv2d/kernel/read, conv2d/bias/read, DMT/_0, DMT/_1, DMT/_2)]]

Caused by op 'conv2d/BiasAdd', defined at:
  File "C:\ProgramData\Anaconda3\envs\tfcndenv\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Progr

/c/Users/A547184/Git/Repos/python-cronicals/TensorFlow/Digit-NN
